In [1]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv("yelp.csv")
# df
texts = df['text']
# texts
stars = df['stars']
# stars

0       My wife took me here on my birthday for breakf...
1       I have no idea why some people give bad review...
2       love the gyro plate. Rice is so good and I als...
3       Rosie, Dakota, and I LOVE Chaparral Dog Park!!...
4       General Manager Scott Petello is a good egg!!!...
5       Quiessence is, simply put, beautiful.  Full wi...
6       Drop what you're doing and drive here. After I...
7       Luckily, I didn't have to travel far to make m...
8       Definitely come for Happy hour! Prices are ama...
9       Nobuo shows his unique talents with everything...
10      The oldish man who owns the store is as sweet ...
11      Wonderful Vietnamese sandwich shoppe. Their ba...
12      They have a limited time thing going on right ...
13      Good tattoo shop. Clean space, multiple artist...
14      I'm 2 weeks new to Phoenix. I looked up Irish ...
15      Was it worth the 21$ for a salad and small piz...
16      We went here on a Saturday afternoon and this ...
17      okay t

In [3]:
from collections import Counter
 
def balance_classes(xs, ys):
    
    freqs = Counter(ys)

    # the least common class is the maximum number we want for all classes
    max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_ys = []
    new_xs = []
    for i, y in enumerate(ys):
        if num_added[y] < max_allowable:
            new_ys.append(y)
            new_xs.append(xs[i])
            num_added[y] += 1
    return new_xs, new_ys

In [4]:
print(Counter(stars))
balanced_x, balanced_y = balance_classes(texts, stars)
print(Counter(balanced_y))

Counter({4: 3526, 5: 3337, 3: 1461, 2: 927, 1: 749})
Counter({5: 749, 4: 749, 2: 749, 3: 749, 1: 749})


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))
t1 = datetime.datetime.now()

# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(balanced_x)
print(datetime.datetime.now() - t1)

from sklearn.feature_extraction.text import TfidfVectorizer
 
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))
t1 = datetime.datetime.now()
 
# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(balanced_x)
print(datetime.datetime.now() - t1)

0:00:02.229075
0:00:02.183851


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_y, test_size=0.33, random_state=42)

In [7]:
from sklearn.svm import LinearSVC

# initialise the SVM classifier
classifier = LinearSVC()

# train the classifier
t1 = datetime.datetime.now()
classifier.fit(X_train, y_train)
print(datetime.datetime.now() - t1)

from sklearn.svm import LinearSVC
 
# initialise the SVM classifier
classifier = LinearSVC()
 
# train the classifier
t1 = datetime.datetime.now()
classifier.fit(X_train, y_train)
print(datetime.datetime.now() - t1)

0:00:00.521719
0:00:00.761426


In [8]:

preds = classifier.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

preds = classifier.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

[2, 3, 3, 1, 2, 1, 1, 4, 5, 4]
[2, 4, 3, 4, 2, 1, 4, 4, 5, 3]
[2, 3, 3, 1, 2, 1, 1, 4, 5, 4]
[2, 4, 3, 4, 2, 1, 4, 4, 5, 3]


In [9]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

0.5186084142394822


In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, preds)

array([[174,  53,   9,   2,   8],
       [ 75,  93,  50,  12,  13],
       [ 30,  41, 105,  49,  29],
       [  7,  10,  43,  94, 100],
       [  8,   2,  14,  40, 175]])